# FORGER

## This is a notebook about General Adversarial Networks. Data from Kaggle : https://www.kaggle.com/ikarus777/best-artworks-of-all-time. 

In [ ]:
import keras
from keras.optimizers import Adam
from keras.layers import Dense, Dropout, Input, BatchNormalization, Reshape, Flatten
from keras.models import Model,Sequential
from tqdm import tqdm
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.layers.advanced_activations import LeakyReLU

from random import shuffle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import time


DATA_PATH = "D:/Mady/Data/130081_310927_bundle_archive/images/images/Vincent_van_Gogh/Vincent_van_Gogh_706.jpg"
RESIZED_PATH = "D:/Mady/Data/130081_310927_bundle_archive/resized/resized/"
OUTPUT_PATH = "D:/Mady/Data/130081_310927_bundle_archive/artists.csv"
SAVED_DATA_PATH = "data/"
RESULTS_DATA_PATH = "results/"

# GETTING THE DATA

In [ ]:
def preprocess_image(img, NEW_WIDTH=128, NEW_HEIGHT=128):
    loaded_img = load_img(img)
    resized_img = loaded_img.resize((NEW_WIDTH, NEW_HEIGHT))
    new_img = np.array(img_to_array(resized_img))
    return new_img

In [ ]:

output_df = pd.read_csv(OUTPUT_PATH)
paintings=[]
outputs=[]
gallery= os.listdir(RESIZED_PATH)
for painting in gallery:
    paintings.append(preprocess_image(RESIZED_PATH+painting))
    outputs.append(output_df[output_df['name'].str.contains(painting.split("_")[0])]["id"].iloc[0])

print(len(paintings))
print(len(outputs))

zipped = list(zip(paintings,outputs))
shuffle(zipped)
paintings, outputs = zip(*zipped)
paintings = np.stack(paintings)

In [ ]:
#Get some information here!!
print(paintings.shape)
print(len(paintings))
#paintings = paintings.reshape(8683, 16384, 3 )
print(paintings.shape)

In [ ]:
#Divide training set 90% of paintings to testing/validation set 10% of paintings
NUM_PAINTINGS = len(paintings)
x_train, x_test = paintings[:round(NUM_PAINTINGS*0.9)], paintings[round(NUM_PAINTINGS*0.9):]
y_train, y_test = outputs[:round(NUM_PAINTINGS*0.9)], outputs[round(NUM_PAINTINGS*0.9):]

In [ ]:
#Get some more information here!!
print(len(x_train))
print(len(x_test))
print(len(y_train))
print(len(y_test))

In [ ]:
print(x_train.shape)

In [ ]:
#serialize the data into folder using np.save and load later using np.load
np.save(SAVED_DATA_PATH+"x_train.npy", x_train)
np.save(SAVED_DATA_PATH+"y_train.npy", y_train)
np.save(SAVED_DATA_PATH+"x_test.npy", x_test)
np.save(SAVED_DATA_PATH+"y_test.npy", y_test)

# VISUALIZING THE DATA

In [ ]:
from keras.datasets import mnist

(x_train, y_train), (x_test,y_test) = mnist.load_data()
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

plt.figure()
plt.title("Random e.g.", fontsize=20)
plt.imshow(x_train[2].astype(int))
print(y_train[2])

In [ ]:

##im1= np.array(img_to_array(load_img(DATA_PATH)))
##im2= np.array(img_to_array(load_img(RESIZED_PATH)))


#im1b= np.array(img_to_array(load_img("D:/Mady/Data/130081_310927_bundle_archive/images/images/Vincent_van_Gogh/Vincent_van_Gogh_708.jpg")))
im2b= "D:/Mady/Data/130081_310927_bundle_archive/resized/resized/Vincent_van_Gogh_707.jpg"


im2b = preprocess_image(im2b, 300, 300)
print(im2b.shape)

plt.figure(1)
plt.title("Random e.g.", fontsize=20)
plt.imshow(im2b.astype(int))



im2b = im2b/256
print(im2b.shape)
im2b.flatten()
print(im2b.shape)
plt.figure(2)
plt.title("Random e.g.2", fontsize=20)
plt.imshow(im2b.astype(int))


# BUILDING THE MODEL

In [ ]:
def adam_optimizer():
    return Adam(lr=0.0002, beta_1=0.5)

In [ ]:
def build_generator(rows,cols,channels):
    model = Sequential()
    #shape of noise = input_shape
    model.add(Dense(256, input_shape=(100,)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    #rows= 128, cols= 128, channels= 3
    model.add(Dense(np.prod((rows,cols,channels)), activation='tanh'))
    model.add(Reshape((rows,cols,channels)))
    
    noise = Input(shape=(100,))
    img = model(noise)
    model = Model(noise,img)
    model.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
    model.summary()
    return model

generator = build_generator(128,128,3)
    
    

In [ ]:
def build_discriminator(rows,cols,channels):
    model = Sequential()
    model.add(Flatten(input_shape=(rows,cols,channels)))
        
    model.add(Dense(1024, input_dim=np.prod((rows,cols,channels)) ))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))
    
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))
    
    model.add(Dense(units=256))
    model.add(LeakyReLU(0.2))

    model.add(Dense(1, activation='sigmoid'))
    
    img = Input(shape=(rows,cols,channels))
    valid = model(img)
    model = Model(img, validity)
    model.compile(loss='binary_crossentropy', optimizer=adam_optimizer(), metrics=['accuracy'])
    model.summary()
    return model

discriminator = build_discriminator(128,128,3)



In [ ]:
class GAN():
    def __init__(self, generator, discriminator):
        self.generator = generator
        self.discriminator = discriminator
        self.gan = self.build_gan()
        
    def build_gan(self):
        self.discriminator.trainable = False
        gan_input = Input(shape=(100,))
        discriminator_input = self.generator(gan_input)
        gan_output = self.discriminator(discriminator_input)
    
        gan = Model(gan_input, gan_output)
        gan.compile(loss='binary_crossentropy', optimizer='adam')
        gan.summary()
        return gan

gan = GAN(generator, discriminator)

# VISUALIZING THE RESULTS

In [ ]:
def plot_generator_images(generator, epoch, examples=100):
    noise = np.random.normal(loc=0, scale=1, size=[examples,100])
    
    generated_images = generator.predict(noise) #.reshape(examples,128,128,3)
    print(generated_images.shape)
    print(generated_images[1].shape)
    print(generated_images[1][1][1])
    #time.sleep(1928132)
    
    
    plt.figure(figsize=(10,10))
    for i in range(examples):
        plt.subplot(10,10,i+1)
        plt.imshow(generated_images[i], interpolation='nearest')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(RESULTS_DATA_PATH+"epoch_"+str(epoch)+" gan_generated_image.png")
    

# TRAINING THE MODEL

In [ ]:
#load the dataset
x_train, y_train = np.load(SAVED_DATA_PATH+"x_train.npy"), np.load(SAVED_DATA_PATH+"y_train.npy")
x_test, y_test = np.load(SAVED_DATA_PATH+"x_test.npy"), np.load(SAVED_DATA_PATH+"y_test.npy")

In [ ]:
def train_model(x_train, y_train, x_test, y_test, gan, epochs=1, batch_size=128):
    
    #how many batches
    batch_count = x_train.shape[0] / batch_size
    
    for epoch in range(1, epochs+1):
        print("Epoch: "+str(epoch))
        for _ in tqdm(range(batch_size)):
            #generate random noise
            noise= np.random.normal(0,1, [batch_size, 100])
            
            #generate fake painting image from noise input
            generated_images = gan.generator.predict(noise)
            
            #get a random set of real images
            #print("OK here is where the issue may be")
            #print(x_train.shape)
            num = np.random.randint(low=0,high=x_train.shape[0],size=batch_size)
            #print(num)
            image_batch = x_train[num]
            
            #construct different batches of real and fake data
            #print(image_batch.shape)
            #print(generated_images.shape)
            x = np.concatenate([image_batch, generated_images])
            
            # Labels for generated and real data
            y_dis=np.zeros(2*batch_size)
            y_dis[:batch_size]=0.9
            
            #Pre train discriminator on  fake and real data  before starting the gan. 
            gan.discriminator.trainable=True
            gan.discriminator.train_on_batch(x, y_dis)
            
            #Tricking the noised input of the Generator as real data
            noise= np.random.normal(0,1, [batch_size, 100])
            y_gen = np.ones(batch_size)
            
            # During the training of gan, 
            # the weights of discriminator should be fixed. 
            #We can enforce that by setting the trainable flag
            discriminator.trainable=False
            
            #training  the GAN by alternating the training of the Discriminator 
            #and training the chained GAN model with Discriminator’s weights freezed.
            print(noise.shape)
            print(y_gen.shape)
            gan.gan.train_on_batch(noise, y_gen)
            
            if (epoch == 20) or (epoch % 20 == 0):
                plot_genrator_images(generator, epoch)
                
                

In [ ]:
train_model(x_train, y_train, x_test, y_test, gan, 100)